In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import metrics
from sklearn import preprocessing
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,label_binarize
from sklearn.metrics import confusion_matrix,accuracy_score,roc_curve,classification_report,auc,roc_auc_score
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
(train_x,train_y),(test_x,test_y) = tf.keras.datasets.cifar10.load_data()

In [ ]:
X = np.concatenate((train_x,test_x))
Y = np.concatenate((train_y,test_y))

train_x, test_x, train_y, test_y = train_test_split(X,Y,test_size=0.2)

train_x = train_x[0:16000]
test_x = test_x[0:4000]
train_y = train_y[0:16000]
test_y = test_y[0:4000]

print("Training Samples : ",train_x.shape[0])
print("Testing Samples : ",test_x.shape[0])

In [ ]:
x_train = np.reshape(train_x, (train_x.shape[0], -1))
x_test = np.reshape(test_x, (test_x.shape[0], -1))

In [ ]:
def Report(train_y,test_y,pred_y,pred_train):

  error = 1 - accuracy_score(train_y,pred_train)
  print("Training error {}".format(error))

  error = 1 - accuracy_score(test_y,pred_y)
  print("Testing error {}".format(error))

  print("\n","Confusion Matrix\n",confusion_matrix(test_y,pred_y))

  print("\n","Classification Report\n",classification_report(test_y,pred_y))


In [ ]:
def plot_ROC(y_test,y_score,x):
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(10):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    plt.figure()

    plt.plot(fpr[2], tpr[2], color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc[2])
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve of '+x)
    plt.legend(loc="lower right")
    plt.show()


In [ ]:
y_train = label_binarize(train_y, classes=[*range(10)])
y_test = label_binarize(test_y, classes=[*range(10)])

In [ ]:
clf = svm.SVC(probability = False, kernel = 'rbf')
clf.fit(x_train,train_y.ravel())

pred_y = clf.predict(x_test)
pred_train = clf.predict(x_train)

Report(train_y,test_y,pred_y,pred_train)

clf = OneVsRestClassifier(svm.SVC(kernel = 'rbf'))
clf.fit(x_train,y_train)
y_score = clf.decision_function(x_test)
plot_ROC(y_test,y_score,'Support Vector Machine')

In [ ]:
n=12
clf = RandomForestClassifier(max_depth=n ,n_estimators=100, criterion = 'entropy' ,random_state=0)
clf.fit(x_train,train_y.ravel())

pred_y = clf.predict(x_test)
pred_train = clf.predict(x_train)

Report(train_y,test_y,pred_y,pred_train)

clf2 = OneVsRestClassifier(RandomForestClassifier(max_depth=n, n_estimators=100, criterion = 'entropy', random_state = 0))
clf2.fit(x_train,y_train)
y_score = clf2.predict_proba(x_test)
plot_ROC(y_test,y_score,'Random Forests')

In [ ]:
clf = DecisionTreeClassifier(random_state=0)
clf.fit(x_train,train_y)

pred_y = clf.predict(x_test)
pred_train = clf.predict(x_train)

Report(train_y,test_y,pred_y,pred_train)

clf3 = OneVsRestClassifier(DecisionTreeClassifier(random_state = 0))
clf3.fit(x_train,y_train)
y_score = clf3.predict_proba(x_test)
plot_ROC(y_test,y_score,'Decision Tree')

In [ ]:
clf = LogisticRegression(random_state=0,max_iter=100000)
clf.fit(x_train,train_y.ravel())

pred_y = clf.predict(x_test)
pred_train = clf.predict(x_train)

Report(train_y,test_y,pred_y,pred_train)

clf4 = OneVsRestClassifier(LogisticRegression(max_iter=100000,random_state=0))
clf4.fit(x_train,y_train)
y_score = clf4.decision_function(x_test)
plot_ROC(y_test,y_score,'Logistic Regression')

In [ ]:
#Support Vector Machine with rbf kernel ML model is best among the given models based on ROC curve analysis.